In [1]:
# Field Calibration data
CM1 = 39200.333  # Initial Conductivity (µS/cm)
TM1 = 17.8       # Initial Temperature (°C)
CM2 = 39182      # Final Conductivity (µS/cm)
TM2 = 17.633     # Final Temperature (°C)

# Estimate Correction Factor (E(t))
# Assuming a linear relationship between initial and final measurements for simplicity
# E1 = CM1 / Cs  # Correction factor for the initial measurement
# E2 = CM2 / Cs  # Correction factor for the final measurement
# E_avg = (E1 + E2) / 2  # Average correction factor
# 
# # Calculate Calibration Corrected Specific Conductance
# # Using the formula: Cscal(t) = E(t) * Cs(t) / (E(t) - Cs(t))
# # Assuming E(t) is the average correction factor E_avg and Cs(t) is the specific conductance calculated previously
# Cscal = E_avg * Cs / (E_avg - Cs)

# Cscal, E_avg


In [2]:
def calculate_specific_conductance_natural_waters(Ye, T):
    Af = -3.17782069713509E-09
    Bf = 4.71041480534068E-07
    Cf = -0.0000332801277284913
    Df = 0.00160578298392272
    Ef = -0.0621932177519881
    Ff = 1.91861301861503
    # 3.2.1. Calculate Specific Conductance at 25 C (Cs) using the first option of Natural Waters correction per EN27888
    # F = Af*T^5 + Bf*T^4 + Cf*T^3 + Df*T^2 + Ef*T + Ff
    # Cs = Ye*F
    F = Af * T**5 + Bf * T**4 + Cf * T**3 + Df * T**2 + Ef * T + Ff
    Cs = Ye * F
    return Cs

In [17]:
def calculate_Rt(CS_t, T):
    C0 = 6.766097E-01
    C1 = 2.005640E-02
    C2 = 1.104259E-04
    C3 = -6.969800E-07
    C4 = 1.003100E-09

    rT = C0 + C1 * T + C2 * T**2 + C3 * T**3 + C4 * T**4
    Rt = CS_t / (rT * 42.914)
    return Rt

def calculate_temp_coefficient(T, S, A, B, C, D, E, F):
    a = A + B * T + C * S + D * T**2 + E * S**2 + F * T * S
    return a

def calculate_salinity(CS_t, T):
    a = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
    b = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
    k = 0.0162
    Rt = calculate_Rt(CS_t, T)
    first_part = sum([a[i] * (Rt ** (0.5 * i)) for i in range(6)])
    second_part = sum([b[i] * (Rt ** (0.5 * i)) for i in range(6)])
    salinity = first_part + (T - 15) * second_part / (1 + k * (T - 15))
    return salinity

def calculate_specific_conductance_seawater(Ye, T):
    A = 1.86221444E+00
    B = 7.99141780E-03
    C = -2.04882760E-03
    D = -4.79386353E-05
    E = 1.67997158E-05
    F = -1.55721008E-05

    S = calculate_salinity(Ye, T)

    a = calculate_temp_coefficient(T, S, A, B, C, D, E, F)
    denominator = 1 - ((25 - T) * a / 100)
    if abs(1 - denominator) < 0.001:
        Cs = Ye / 0.001
    else:
        Cs = Ye / denominator
    return Cs

In [4]:
def calculate_specific_conductance_entered_temp_coeff(Ye, T, a):
    Cs = Ye / (1 - ((25 - T) * a / 100))
    return Cs


In [5]:
def calculate_specific_conductance_Ye(Ye, T, method):
    """
    Calculate specific conductance based on the selected method.
    """
    if method == "Natural Waters":
        # Implement the Natural Waters correction formula
        return calculate_specific_conductance_natural_waters(Ye, T)
    elif method == "Seawater Compensation":
        # Implement the Seawater Compensation formula
        pass
    elif method == "Entered Temperature Coefficient":
        # Implement the Entered Temperature Coefficient formula
        pass
    else:
        raise ValueError("Invalid method")

    return

In [6]:
def calculate_calibration_correction(CS1, MS1, CS2, MS2, t1, t2, t):
    # Ensure resistance values don't cause division by zero
    R1 = (1 / CS1 - 1 / MS1) if abs(1 / CS1 - 1 / MS1) >= 1E-11 else 1E-11
    R2 = (1 / CS2 - 1 / MS2) if abs(1 / CS2 - 1 / MS2) >= 1E-11 else 1E-11

    # Calculate the slope (A) and intercept (B) for linear interpolation
    A = (R2 - R1) / (t2 - t1)
    B = R1 - (A * t1)

    # Calculate resistance at time t
    if t < t1:
        R_t = R1
    elif t > t2:
        R_t = R2
    else:
        R_t = A * t + B

    # Ensure R_t doesn't cause division by zero
    if abs(R_t) < 1E-11:
        R_t = 1E-11

    E_t = 1 / R_t

    # Calculate the calibration corrected specific conductance
    Cscal_t = E_t * CS1 / (E_t - CS1)
    return Cscal_t

In [11]:
def calculate_salinity(CS_t, T, a, b, k):
    Rt = calculate_Rt(CS_t, T)
    first_part = sum([a[i] * (Rt ** (0.5 * i)) for i in range(6)])
    second_part = sum([b[i] * (Rt ** (0.5 * i)) for i in range(6)])
    salinity = first_part + (T - 15) * second_part / (1 + k * (T - 15))
    return salinity


In [ ]:
a = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
b = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
k = 0.0162
salinity = calculate_salinity(Cscal, T, a, b, k)

In [25]:
# Example usage
Ye = 30697.7  # Example Electrical Conductivity
T = 21.81  # Example temperature
method = "Natural Waters"  # Example method

# Calculate specific conductance
Cs = calculate_specific_conductance_Ye(Ye, T, method)

In [26]:
def calculate_MS(CM, TM, S):
    """
    Calculate MS (Measured Specific Conductance) using Seawater Compensation method.

    Parameters:
    CM (float): Measured Conductivity (uS/cm)
    TM (float): Measured Temperature (degrees C)
    S (float): Salinity (if known, otherwise an estimated value)

    Returns:
    float: The calculated MS value
    """
    return calculate_specific_conductance_seawater(CM, TM, S)


In [27]:
# Assume calibration points and times
CS1, MS1, CS2, MS2 = 0, 0, 0, 0  # Example values
t1, t2, t = 0, 0, 0  # Example times

# Calculate calibration correction
Cscal = calculate_calibration_correction(CS1, MS1, CS2, MS2, t1, t2, t)

ZeroDivisionError: division by zero

In [28]:
# Recalculate salinity
a = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
b = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
k = 0.0162
salinity = calculate_salinity(Cscal, T, a, b, k)

NameError: name 'Cscal' is not defined

In [ ]:
def calculate_MS(CM, TM, S):
    return calculate_specific_conductance_seawater(CM, TM, S)

# Experiment

In [7]:
import pandas as pd

# con_data = pd.read_csv('../data/W12_Con_7_6_2023_lkg.csv')
con_data = pd.read_csv('/Users/christina/scc_db/data/RB1_ConDL_6_9_2023_AH.csv')

In [8]:
AC_all = con_data['HighRange']

In [9]:
con_data.iloc[8]

#                                                          9
Date Time, GMT-04:00                    06/02/23 11:20:00 AM
LowRange                                             33594.4
HighRange                                            30697.7
temp                                                   21.81
Coupler Detached (LGR S/N: 20636185)                     NaN
Coupler Attached (LGR S/N: 20636185)                     NaN
Name: 8, dtype: object

In [8]:
def calculate_Rt(CS_t, T):
    C0 = 6.766097E-01
    C1 = 2.005640E-02
    C2 = 1.104259E-04
    C3 = -6.969800E-07
    C4 = 1.003100E-09

    rT = C0 + C1 * T + C2 * T**2 + C3 * T**3 + C4 * T**4
    Rt = CS_t / (rT * 42.914)
    return Rt

def calculate_temp_coefficient(T, S, A, B, C, D, E, F):
    a = A + B * T + C * S + D * T**2 + E * S**2 + F * T * S
    return a

def calculate_salinity(CS_t, T):
    a = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
    b = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
    k = 0.0162
    Rt = calculate_Rt(CS_t, T)
    first_part = sum([a[i] * (Rt ** (0.5 * i)) for i in range(6)])
    second_part = sum([b[i] * (Rt ** (0.5 * i)) for i in range(6)])
    salinity = first_part + (T - 15) * second_part / (1 + k * (T - 15))
    return salinity

def calculate_specific_conductance_seawater(Ye, T):
    A = 1.86221444E+00
    B = 7.99141780E-03
    C = -2.04882760E-03
    D = -4.79386353E-05
    E = 1.67997158E-05
    F = -1.55721008E-05

    S = calculate_salinity(Ye, T) / 1000000
#     S = 29.4197
    print("Salinity:", S)

    a = calculate_temp_coefficient(T, S, A, B, C, D, E, F)
    
    denominator = 1 - ((25 - T) * a / 100)
    if abs(1 - denominator) < 0.001:
        Cs = Ye / 0.001
    else:
        Cs = Ye / denominator
    return Cs

calculate_specific_conductance_seawater(30697.7, 21.81)

Salinity: 22.46523436754503


32754.553637234487

In [2]:
calculate_specific_conductance_seawater(30697.7, 21.81)

Salinity: 22465234.36754503


-0.00011349952975220326

In [21]:
a = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
b = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
k = 0.0162
salinity = calculate_salinity(30697.7, 21.81, a, b, k)

TypeError: calculate_salinity() takes 2 positional arguments but 5 were given

In [37]:
salinity

22465234.36754503

In [42]:

# Measured calibration data
CS1 = 39200.333  # Initial Conductivity (µS/cm)
t1 = 17.8       # Initial Temperature (°C)
CS2 = 39182      # Final Conductivity (µS/cm)
t2 = 17.633     # Final Temperature (°C)
MS1 = 29.49
MS2 = 29.63
t = 21.81
# Calculate MS1 and MS2
# MS1 = calculate_MS(CM1, TM1, S)
# MS2 = calculate_MS(CM2, TM2, S)

calculate_calibration_correction(CS1, MS1, CS2, MS2, t1, t2, t)

29.63001047901926

In [44]:
a = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
b = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
k = 0.0162
salinity = calculate_salinity(29.63001047901926, T, a, b, k)

In [45]:
salinity

19.67930518485545